Create 'database' of road stretches to assign accident probabilities

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

sns.set(color_codes=True)
sns.set_context(rc = {'patch.linewidth': 2.0})
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('seaborn-white')

In [2]:
def date_columns(query):
    """If a date column is included in the query, parse it as a date in the
    dataframe."""
    dates = []
    fields = ["Collision_Date", "Process_Date"]
    if '*' in query:
        dates = fields
    else:
        for date in fields:
            if date in query:
                dates.append(date)

        if not dates:
            dates = None

    return dates

In [6]:
def run_sql(query, sql_file="../data/switrs.sqlite3"):
    with sqlite3.connect(sql_file) as con:
        df = pd.read_sql_query(query, con, parse_dates=date_columns(query),)

        return df

In [4]:
def set_factorize(df, in_name, out_name = '', show = False):
    if out_name is '':
        out_name = in_name
        df[in_name + '_old'] = df[in_name]
    
    vals, cols = df[in_name].factorize()
    
    df[in_name] = vals
    
    if show:
        print df[in_name].value_counts()

In [7]:
query = """
SELECT * FROM Collision
WHERE Longitude IS NOT NULL 
AND Latitude IS NOT NULL 
AND Primary_Road IS NOT NULL
AND Collision_Time IS NOT NULL
AND Caltrans_County IS NOT NULL
"""

df_loc = run_sql(query)

In [8]:
df_loc['Collision_Year']      = df_loc.Collision_Date.dt.year
df_loc['Collision_Month']     = df_loc.Collision_Date.dt.month
df_loc['Collision_Day']       = df_loc.Collision_Date.dt.day
df_loc['Collision_DayOfWeek'] = df_loc.Collision_Date.dt.dayofweek 

df_loc['Collision_Hour']   = df_loc.Collision_Time.str.split(':', n=3, expand=True).apply(pd.Series).astype(int)[0]
df_loc['Collision_Minute'] = df_loc.Collision_Time.str.split(':', n=3, expand=True).apply(pd.Series).astype(int)[1]

df_loc['Collision_Hours']  = df_loc.Collision_Hour + df_loc.Collision_Minute / 60.0
df_loc['Collision_Minutes'] = 60 * df_loc.Collision_Hour + df_loc.Collision_Minute

df_loc['Postmile_Code'] = df_loc.Postmile_Prefix + df_loc.Postmile.astype(str)

In [ ]:
df_highway = df_loc[(~df_loc.Postmile_Code.isnull()) & (~df_loc.Caltrans_County.isnull())]

In [80]:
s = df_highway.State_Route.value_counts()

print sorted(s[s > 1000].index)

[1.0, 4.0, 5.0, 8.0, 10.0, 14.0, 15.0, 22.0, 24.0, 40.0, 41.0, 50.0, 55.0, 57.0, 58.0, 60.0, 80.0, 85.0, 91.0, 92.0, 99.0, 101.0, 105.0, 118.0, 120.0, 126.0, 134.0, 170.0, 198.0, 205.0, 210.0, 215.0, 280.0, 580.0, 605.0, 680.0]


In [89]:
cols = ['Case_ID', 'Caltrans_County', 'Postmile_Code', 'State_Route', 'Direction']

df_highway[cols][df_highway.State_Route == 280].sample(20)

,Case_ID,Caltrans_County,Postmile_Code,State_Route,Direction
737535,6144131,SF,R1.45,280.0,S
668272,6203810,SCL,R2.02,280.0,N
364066,5140735,SF,R5.63,280.0,N
263152,4834645,SCL,R2.365,280.0,N
429585,5325620,SCL,R0.44,280.0,N
361036,5131989,SCL,R3.28,280.0,N
889559,6595425,SCL,L5.03,280.0,N
484659,5499992,SM,M27.11,280.0,S
740521,6157150,SCL,R2.71,280.0,N
804319,6365932,SF,R6.62,280.0,N


In [79]:
for c in df_loc.columns:
    print c

Case_ID
Jurisdiction
Officer_ID
Reporting_District
CHP_Shift
Population
County_City_Location
Special_Condition
Beat_Type
CHP_Beat_Type
City_Division_LAPD
CHP_Beat_Class
Beat_Number
Primary_Road
Secondary_Road
Distance
Direction
Intersection
Weather_1
Weather_2
State_Highway_Indicator
Caltrans_County
Caltrans_District
State_Route
Route_Suffix
Postmile_Prefix
Postmile
Location_Type
Ramp_Intersection
Side_Of_Highway
Tow_Away
Collision_Severity
Killed_Victims
Injured_Victims
Party_Count
Primary_Collision_Factor
PCF_Violation_Code
PCF_Violation_Category
PCF_Violation
PCF_Violation_Subsection
Hit_And_Run
Type_Of_Collision
Motor_Vehicle_Involved_With
Ped_Action
Road_Surface
Road_Condition_1
Road_Condition_2
Lighting
Control_Device
CHP_Road_Type
Pedestrian_Collision
Bicycle_Collision
Motorcycle_Collision
Truck_Collision
Not_Private_Property
Alcohol_Involved
Statewide_Vehicle_Type_At_Fault
CHP_Vehicle_Type_At_Fault
Severe_Injury_Count
Other_Visible_Injury_Count
Complaint_Of_Pain_Injury_Count
Pe